# Bước 2a: Tính Ma trận Whitening (v2 - inner_content) và lưu dữ liệu Mahalanobis


## Mục tiêu của script
Script này dùng để:
- **Tải database vector train** đã được gộp sẵn (tensor dạng `(N_samples, NUM_LAYERS, D_FEATURES)`).
- Với **mỗi layer (1 → 12)**:
  - Trích xuất ma trận đặc trưng `X_layer` có shape `(N, D)` (ở đây `D = 2304`).
  - Tính:
    - **vector trung bình** `mean` (kích thước `D`)
    - **ma trận whitening** `W = Σ^{-1/2}` (kích thước `D x D`)
- Lưu toàn bộ kết quả ra file `.pt` để dùng cho các bước sau (ví dụ khoảng cách Mahalanobis).

## Thư viện sử dụng
- `torch`: xử lý tensor, SVD, tính covariance, chạy GPU nếu có.
- `os`: thao tác đường dẫn file/thư mục.
- `numpy`: import sẵn (trong code hiện tại **chưa dùng**).
- `tqdm.auto`: hiển thị progress bar khi tính từng layer.

## Cấu hình tham số
- `NUM_LAYERS = 12`  
  Số layer cần tính (layer_1 → layer_12).

- `REGULARIZATION = 1e-5`  
  Hằng số **jitter** cộng vào đường chéo ma trận hiệp phương sai để:
  - tránh ma trận suy biến (singular)
  - giúp SVD ổn định hơn
  - đảm bảo khả nghịch khi tính `Σ^{-1/2}`

- `D_FEATURES = 2304`  
  Kích thước vector đặc trưng (embedding dimension).

- `device = cuda/cpu`  
  Nếu có GPU → dùng CUDA để tăng tốc tính toán.

## Thiết lập đường dẫn I/O
- `drive_base_path`: thư mục gốc dự án trên Google Drive.
- `db_cache_dir`: thư mục cache lưu các file `.pt`.

### Input
- `combined_train_db_vectors_v2_inner_content.pt`  
  Chứa tensor `db_vectors` shape:
  - `(N_samples, NUM_LAYERS, D_FEATURES)`

### Output
- `mahalanobis_data_v2_inner_content.pt`  
  Lưu:
  - `means[layer_i]`
  - `whitening_matrices[layer_i]`

## Hàm `calculate_whitening_matrix(X, reg)`

### Mục đích
Tính:
- `mean`: vector trung bình theo feature
- `whitening_matrix`: ma trận làm trắng `W = Σ^{-1/2}`

Trong đó:
- `X` có shape `(N, D)`
- `Σ` là ma trận hiệp phương sai của dữ liệu đã center

### Bước 1 — Kiểm tra số mẫu
- Lấy `N, D` từ `X.shape`
- Nếu `N <= 1` → không đủ mẫu để tính covariance `(N-1)` → trả về `None`

### Bước 2 — Tính vector trung bình (mean)
- `mean = X.mean(dim=0)`
- `mean` có shape `(D,)`

### Bước 3 — Center dữ liệu
- `X_c = X - mean`
- Mục đích: đưa dữ liệu về trung bình 0 trước khi tính covariance

### Bước 4 — Tính ma trận hiệp phương sai (Covariance)
- Công thức:
  - `Sigma = (X_c.T @ X_c) / (N - 1)`
- `Sigma` có shape `(D, D)`

### Bước 5 — Regularization (jitter)
- Cộng `reg * I` vào đường chéo:
  - `Sigma += I * reg`
- Mục đích:
  - tránh eigenvalue ≈ 0
  - tăng ổn định cho SVD
  - giúp ma trận khả nghịch tốt hơn

### Bước 6 — Tính whitening matrix bằng SVD
- Thực hiện:
  - `U, S, Vh = svd(Sigma)`
- Tạo:
  - `S_inv_sqrt = diag(1/sqrt(S))`
- Tính:
  - `W = Vh.T @ S_inv_sqrt @ U.T`

Kết quả:
- `W` có shape `(D, D)`
- Trả về `mean` và `W` (đưa về CPU để lưu)

## Luồng thực thi chính (`__main__`)

### Bước 1 — Load database vectors
- Load file:
  - `db_vectors = torch.load(db_vectors_file)`
- Lấy shape:
  - `(N_samples, n_layers, D_features_loaded)`
- Kiểm tra:
  - `n_layers == NUM_LAYERS`
  - `D_features_loaded == D_FEATURES`

### Bước 2 — Khởi tạo cấu trúc lưu kết quả
Tạo dict:
- `mahalanobis_data['means']`: lưu mean từng layer
- `mahalanobis_data['whitening_matrices']`: lưu whitening matrix từng layer

Key đặt theo format:
- `layer_1`, `layer_2`, ..., `layer_12`

### Bước 3 — Tính toán theo từng layer (1 → 12)
Với mỗi `i` trong `range(NUM_LAYERS)`:
- Đặt tên:
  - `layer_name = f"layer_{i+1}"`

#### 3.1 Trích dữ liệu layer
- `X_layer = db_vectors[:, i, :]`
- Shape:
  - `(N_samples, 2304)`
- Đưa lên GPU (nếu có):
  - `.to(device)`

#### 3.2 Tính mean và whitening
- Gọi:
  - `mean, whitening_matrix = calculate_whitening_matrix(X_layer, REGULARIZATION)`

#### 3.3 Lưu kết quả nếu thành công
- Nếu `mean` và `whitening_matrix` không phải `None`:
  - `means[layer_name] = mean`
  - `whitening_matrices[layer_name] = whitening_matrix`
- Nếu lỗi:
  - bỏ qua layer đó và in cảnh báo

#### 3.4 Giải phóng bộ nhớ GPU
- `del X_layer`
- `torch.cuda.empty_cache()`

### Bước 4 — Lưu kết quả ra file output
- Dùng:
  - `torch.save(mahalanobis_data, output_file)`

Output chứa:
- `means[layer_k]` có shape `(2304,)`
- `whitening_matrices[layer_k]` có shape `(2304, 2304)`


## Kết quả đầu ra
Sau khi chạy xong, script tạo file:
- `mahalanobis_data_v2_inner_content.pt`

File này dùng cho các bước sau như:
- Whitening vector theo từng layer
- Tính khoảng cách Mahalanobis
- So sánh embedding ổn định hơn (giảm tương quan giữa các chiều)

In [ ]:
# Import các thư viện cần thiết
import torch
import os
import numpy as np
from tqdm.auto import tqdm

print("--- Bắt đầu Bước 2a: Tính Ma trận Whitening (v2 - inner_content) ---")

# --- 1. CẤU HÌNH ---
NUM_LAYERS = 12 # 12 lớp (layer_1 đến layer_12)
REGULARIZATION = 1e-5 # Hằng số jitter để đảm bảo ma trận khả nghịch
D_FEATURES = 2304 # Kích thước vector

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Sử dụng thiết bị: {device}")

# --- 2. THIẾT LẬP CÁC ĐƯỜNG DẪN ---
drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep'
db_cache_dir = os.path.join(drive_base_path, 'cached_databases')

# --- INPUT: Database Train (v2 - inner content) ---
db_vectors_file = os.path.join(db_cache_dir, 'combined_train_db_vectors_v2_inner_content.pt')

# --- OUTPUT: File chứa các ma trận Mahalanobis ---
output_file = os.path.join(db_cache_dir, 'mahalanobis_data_v2_inner_content.pt') # Tên file mới

print(f"File database input: {db_vectors_file}")
print(f"File ma trận output: {output_file}")

# --- 3. HÀM TÍNH TOÁN ---

def calculate_whitening_matrix(X, reg):
    """
    Tính toán vector trung bình (mean) và ma trận làm trắng (whitening matrix W = Σ⁻¹/²)
    cho một tập dữ liệu X (N, D).
    """
    N, D = X.shape
    if N <= 1:
        print("    CẢNH BÁO: Không đủ mẫu (N<=1) để tính hiệp phương sai.")
        return None, None

    # 1. Tính vector trung bình
    mean = X.mean(dim=0)

    # 2. Căn giữa dữ liệu (Center the data)
    X_c = X - mean

    # 3. Tính ma trận hiệp phương sai (Covariance Matrix)
    # Sigma = (1/N-1) * X_c.T @ X_c
    Sigma = (X_c.T @ X_c) / (N - 1)

    # 4. Thêm jitter (regularization) để đảm bảo ma trận khả nghịch
    Sigma += torch.eye(D, device=device) * reg

    # 5. Tính ma trận làm trắng (W = Σ⁻¹/²) dùng SVD
    try:
        U, S, Vh = torch.linalg.svd(Sigma)
        S_inv_sqrt = torch.diag(1.0 / torch.sqrt(S))
        # W = Vh.T @ S_inv_sqrt @ U.T
        whitening_matrix = (Vh.T @ S_inv_sqrt @ U.T)
        return mean.cpu(), whitening_matrix.cpu()

    except Exception as e:
        print(f"    LỖI: Không thể tính SVD hoặc ma trận làm trắng: {e}")
        return None, None

# --- 4. THỰC THI CHƯƠNG TRÌNH ---
if __name__ == "__main__":
    print("\nĐang tải database...")
    try:
        db_vectors = torch.load(db_vectors_file)
        N_samples, n_layers, D_features_loaded = db_vectors.shape
        print(f" -> Tải thành công! Shape: ({N_samples}, {n_layers}, {D_features_loaded})")
        assert n_layers == NUM_LAYERS and D_features_loaded == D_FEATURES
    except Exception as e:
        print(f"LỖI: Không thể tải file database '{db_vectors_file}': {e}")
        exit()

    mahalanobis_data = {
        'means': {},
        'whitening_matrices': {}
    }

    print(f"\nBắt đầu tính toán {NUM_LAYERS} ma trận...")
    for i in tqdm(range(NUM_LAYERS), desc="Tính toán ma trận"):
        layer_name = f"layer_{i+1}"
        # pbar.write(f"  Đang xử lý {layer_name}...")

        try:
            X_layer = db_vectors[:, i, :].to(device) # Gửi [N, 2304] lên GPU
            mean, whitening_matrix = calculate_whitening_matrix(X_layer, REGULARIZATION)

            if mean is not None and whitening_matrix is not None:
                mahalanobis_data['means'][layer_name] = mean
                mahalanobis_data['whitening_matrices'][layer_name] = whitening_matrix
            else:
                print(f"    -> BỎ QUA {layer_name} do lỗi tính toán.")

            del X_layer
            torch.cuda.empty_cache()

        except Exception as e:
            print(f"    LỖI nghiêm trọng khi xử lý {layer_name}: {e}")

    # --- Lưu kết quả ---
    print("\nĐang lưu kết quả...")
    try:
        torch.save(mahalanobis_data, output_file)
        print(f" -> Đã lưu thành công 12 ma trận (mean và whitening) vào:\n    {output_file}")
    except Exception as e:
        print(f"LỖI: Không thể lưu file output: {e}")

    print("\n--- BƯỚC 2a (TÍNH MA TRẬN) HOÀN TẤT! ---")

--- Bắt đầu Bước 2a: Tính Ma trận Whitening (v2 - inner_content) ---
Sử dụng thiết bị: cpu
File database input: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/cached_databases/combined_train_db_vectors_v2_inner_content.pt
File ma trận output: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/cached_databases/mahalanobis_data_v2_inner_content.pt

Đang tải database...
 -> Tải thành công! Shape: (15521, 12, 2304)

Bắt đầu tính toán 12 ma trận...


Tính toán ma trận:   0%|          | 0/12 [00:00<?, ?it/s]


Đang lưu kết quả...
 -> Đã lưu thành công 12 ma trận (mean và whitening) vào:
    /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/cached_databases/mahalanobis_data_v2_inner_content.pt

--- BƯỚC 2a (TÍNH MA TRẬN) HOÀN TẤT! ---
